## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Domino Data Lab Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. You need to make sure that these datasets are uploaded and present in the same directory as this notebook file, since we will be importing these files in Spark and create the following tables under the __`country_club`__ database using Spark SQL.

1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into Spark to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the Domino platform, but for the final deliverable, please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

# Checking Existence of Spark Environment Variables

Make sure your notebook is loaded using a PySpark Workspace. If you open up a regular Jupyter workspace the following variables might not exist

In [1]:
spark

NameError: name 'spark' is not defined

In [2]:
sqlContext

In [3]:
!pip install py4j

    100% |████████████████████████████████| 204kB 22.4MB/s ta 0:00:01
pyspark 2.4.0 has requirement py4j==0.10.7, but you'll have py4j 0.10.8.1 which is incompatible.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Run the following if you failed to open a notebook in the PySpark Workspace

This will work assuming you are using Spark in the cloud on domino or you might need to configure with your own spark instance if you are working offline

In [4]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

# Create a utility function to run SQL commands

Instead of typing the same python functions repeatedly, we build a small function where you can just pass your query to get results.

- Remember we are using Spark SQL in PySpark
- We can't run multiple SQL statements in one go (no semi-colon ';' separated SQL statements)
- We can run multi-line SQL queries (but still has to be a single statement)

In [5]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

# Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [6]:
run_sql('drop database if exists country_club cascade')
run_sql('create database country_club')
dbs = run_sql('show databases')
dbs.toPandas()

databaseName
0  country_club
1       default

# Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you.

To get started, first make sure you have already uploaded the three CSV files and they are present in the same directory as the notebook.

Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [7]:
# File location and type
file_location_bookings = "./Bookings.csv"
file_location_facilities = "./Facilities.csv"
file_location_members = "./Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [8]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



# Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [14]:
!rm -rf spark-warehouse
!ls

Bookings.csv  Facilities.csv  metastore_db
derby.log     Members.csv     Mini_Project_SQL_with_Spark.ipynb


In [15]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [16]:
run_sql('use country_club')
run_sql('REFRESH table bookings')
run_sql('REFRESH table facilities')
run_sql('REFRESH table members')
tbls = run_sql('show tables')
tbls.toPandas()

database   tableName  isTemporary
0  country_club    bookings        False
1  country_club  facilities        False
2  country_club     members        False

# Test a sample SQL query

__Note:__ You can use multi-line SQL queries (but still a single statement) as follows

In [17]:
result = run_sql('''
                    SELECT * 
                    FROM bookings 
                    LIMIT 3
                 ''')
result.toPandas()

bookid  facid  memid           starttime  slots
0       0      3      1 2012-07-03 11:00:00      2
1       1      4      1 2012-07-03 08:00:00      2
2       2      6      0 2012-07-03 18:00:00      2

# Your Turn: Solve the following questions with Spark SQL

- Make use of the `run_sql(...)` function as seen in the previous example
- You can write multi-line SQL queries but it has to be a single statement (no use of semi-colons ';')
- Make use of the `toPandas()` function as depicted in the previous example to display the query results

#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [22]:
result = run_sql('''
                   SELECT * FROM facilities where membercost <> 0 
                 ''')
result.toPandas()

facid            name  membercost  guestcost  initialoutlay  \
0      0  Tennis Court 1         5.0       25.0          10000   
1      1  Tennis Court 2         5.0       25.0           8000   
2      4  Massage Room 1         9.9       80.0           4000   
3      5  Massage Room 2         9.9       80.0           4000   
4      6    Squash Court         3.5       17.5           5000   

   monthlymaintenance  
0                 200  
1                 200  
2                3000  
3                3000  
4                  80

####  Q2: How many facilities do not charge a fee to members?

In [27]:
result = run_sql('''
                   SELECT * FROM facilities where membercost = 0 
                 ''')
result.toPandas()

facid             name  membercost  guestcost  initialoutlay  \
0      2  Badminton Court         0.0       15.5           4000   
1      3     Table Tennis         0.0        5.0            320   
2      7    Snooker Table         0.0        5.0            450   
3      8       Pool Table         0.0        5.0            400   

   monthlymaintenance  
0                  50  
1                  10  
2                  15  
3                  15

#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [30]:
result = run_sql('''
                   SELECT facid,  name, membercost, monthlymaintenance,0.2 * monthlymaintenance as m FROM facilities
                   where (membercost <> 0) and (membercost < 0.2 * monthlymaintenance)
                ''')
result.toPandas()

facid            name  membercost  monthlymaintenance      m
0      0  Tennis Court 1         5.0                 200   40.0
1      1  Tennis Court 2         5.0                 200   40.0
2      4  Massage Room 1         9.9                3000  600.0
3      5  Massage Room 2         9.9                3000  600.0
4      6    Squash Court         3.5                  80   16.0

#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [31]:
result = run_sql('''
                    SELECT * FROM facilities where facid =1 or facid =5
                ''') 
result.toPandas()

facid            name  membercost  guestcost  initialoutlay  \
0      1  Tennis Court 2         5.0       25.0           8000   
1      5  Massage Room 2         9.9       80.0           4000   

   monthlymaintenance  
0                 200  
1                3000

#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [34]:
result = run_sql('''
                   SELECT name,monthlymaintenance,
                   CASE WHEN monthlymaintenance > 100 THEN "expensive"
                   ELSE "cheap" END AS label
                   FROM facilities
                 ''')
result.toPandas()

name  monthlymaintenance      label
0   Tennis Court 1                 200  expensive
1   Tennis Court 2                 200  expensive
2  Badminton Court                  50      cheap
3     Table Tennis                  10      cheap
4   Massage Room 1                3000  expensive
5   Massage Room 2                3000  expensive
6     Squash Court                  80      cheap
7    Snooker Table                  15      cheap
8       Pool Table                  15      cheap

#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [88]:
result = run_sql('''
                    SELECT firstname, surname FROM members 
                    WHERE joindate = (
                    SELECT MAX(joindate) 
                    FROM Members)
                 ''')
result.toPandas()

firstname surname
0    Darren   Smith

####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [92]:
result = run_sql('''
                    SELECT sub.court, CONCAT( sub.firstname,  ' ', sub.surname ) AS fullname
                    FROM (
                    SELECT Facilities.name AS court, Members.firstname AS firstname, Members.surname AS surname
                    FROM Bookings
                    INNER JOIN Facilities ON Bookings.facid = Facilities.facid
                    AND Facilities.name LIKE  'Tennis Court%'
                    INNER JOIN Members ON Bookings.memid = Members.memid
                    ) sub
                    GROUP BY sub.court, sub.firstname, sub.surname
                    ORDER BY fullname
                 ''')
result.toPandas()

court           fullname
0   Tennis Court 2         Anne Baker
1   Tennis Court 1         Anne Baker
2   Tennis Court 1       Burton Tracy
3   Tennis Court 2       Burton Tracy
4   Tennis Court 1       Charles Owen
5   Tennis Court 2       Charles Owen
6   Tennis Court 2       Darren Smith
7   Tennis Court 2      David Farrell
8   Tennis Court 1      David Farrell
9   Tennis Court 1        David Jones
10  Tennis Court 2        David Jones
11  Tennis Court 1       David Pinker
12  Tennis Court 1      Douglas Jones
13  Tennis Court 1      Erica Crumpet
14  Tennis Court 2     Florence Bader
15  Tennis Court 1     Florence Bader
16  Tennis Court 1        GUEST GUEST
17  Tennis Court 2        GUEST GUEST
18  Tennis Court 2     Gerald Butters
19  Tennis Court 1     Gerald Butters
20  Tennis Court 2   Henrietta Rumney
21  Tennis Court 1         Jack Smith
22  Tennis Court 2         Jack Smith
23  Tennis Court 2    Janice Joplette
24  Tennis Court 1    Janice Joplette
25  Tennis Court 1     Jemima Farrell
26  Tennis Court 2     Jemima Farrell
27  Tennis Court 1        Joan Coplin
28  Tennis Court 1          John Hunt
29  Tennis Court 2          John Hunt
30  Tennis Court 1    Matthew Genting
31  Tennis Court 2  Millicent Purview
32  Tennis Court 1         Nancy Dare
33  Tennis Court 2         Nancy Dare
34  Tennis Court 1    Ponder Stibbons
35  Tennis Court 2    Ponder Stibbons
36  Tennis Court 1   Ramnaresh Sarwin
37  Tennis Court 2   Ramnaresh Sarwin
38  Tennis Court 1         Tim Boothe
39  Tennis Court 2         Tim Boothe
40  Tennis Court 2         Tim Rownam
41  Tennis Court 1         Tim Rownam
42  Tennis Court 1      Timothy Baker
43  Tennis Court 2      Timothy Baker
44  Tennis Court 2        Tracy Smith
45  Tennis Court 1        Tracy Smith

#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [93]:
result = run_sql('''
                   SELECT Facilities.name AS facility, Members.firstname, Members.surname,
                   CASE
                   WHEN Bookings.memid =0
                   THEN Facilities.guestcost * Bookings.slots
                   ELSE Facilities.membercost * Bookings.slots
                   END AS cost
                   FROM Bookings
                   INNER JOIN Facilities ON Bookings.facid = Facilities.facid
                   AND Bookings.starttime LIKE  '2012-09-14%'
                   AND (((Bookings.memid = 0) AND (Facilities.guestcost * Bookings.slots > 30))
                   OR ((Bookings.memid != 0) AND (Facilities.membercost * Bookings.slots > 30)))
                   INNER JOIN Members ON Bookings.memid = Members.memid
                   ORDER BY cost DESC
                 ''')
result.toPandas()

facility firstname  surname   cost
0   Massage Room 2     GUEST    GUEST  320.0
1   Massage Room 1     GUEST    GUEST  160.0
2   Massage Room 1     GUEST    GUEST  160.0
3   Massage Room 1     GUEST    GUEST  160.0
4   Tennis Court 2     GUEST    GUEST  150.0
5   Tennis Court 1     GUEST    GUEST   75.0
6   Tennis Court 1     GUEST    GUEST   75.0
7   Tennis Court 2     GUEST    GUEST   75.0
8     Squash Court     GUEST    GUEST   70.0
9   Massage Room 1    Jemima  Farrell   39.6
10    Squash Court     GUEST    GUEST   35.0
11    Squash Court     GUEST    GUEST   35.0

#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [104]:
result = run_sql('''
                    SELECT Facilities.name AS facility, Members.firstname, Members.surname,
                   CASE
                   WHEN Bookings.memid =0
                   THEN Facilities.guestcost * Bookings.slots
                   ELSE Facilities.membercost * Bookings.slots
                   END AS cost
                   FROM Bookings,Facilities,Members where
                   Bookings.facid = Facilities.facid
                   AND Bookings.starttime LIKE  '2012-09-14%'
                   AND (((Bookings.memid = 0) AND (Facilities.guestcost * Bookings.slots > 30))
                   OR ((Bookings.memid != 0) AND (Facilities.membercost * Bookings.slots > 30)))
                   AND Bookings.memid = Members.memid
                   ORDER BY cost DESC
                 ''')
result.toPandas()

facility firstname  surname   cost
0   Massage Room 2     GUEST    GUEST  320.0
1   Massage Room 1     GUEST    GUEST  160.0
2   Massage Room 1     GUEST    GUEST  160.0
3   Massage Room 1     GUEST    GUEST  160.0
4   Tennis Court 2     GUEST    GUEST  150.0
5   Tennis Court 1     GUEST    GUEST   75.0
6   Tennis Court 1     GUEST    GUEST   75.0
7   Tennis Court 2     GUEST    GUEST   75.0
8     Squash Court     GUEST    GUEST   70.0
9   Massage Room 1    Jemima  Farrell   39.6
10    Squash Court     GUEST    GUEST   35.0
11    Squash Court     GUEST    GUEST   35.0

#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [102]:
result = run_sql('''
                    SELECT sub.facility, (sub.cost) as revenue FROM
                    (SELECT Facilities.name as facility,
                    CASE
                    WHEN Bookings.memid = 0
                    THEN Facilities.guestcost * Bookings.slots
                    ELSE Facilities.membercost * Bookings.slots
                    END AS cost
                    FROM Bookings
                    INNER JOIN Facilities ON Bookings.facid = Facilities.facid
                    INNER JOIN Members ON Bookings.memid = Members.memid)sub 
                    group by sub.facility,sub.cost order by sub.cost desc
                 ''') 
result.toPandas()

facility  revenue
0    Massage Room 1    640.0
1    Massage Room 1    480.0
2    Massage Room 2    320.0
3    Massage Room 1    320.0
4    Tennis Court 1    300.0
5      Squash Court    245.0
6    Tennis Court 1    225.0
7    Tennis Court 2    225.0
8    Massage Room 2    160.0
9    Massage Room 1    160.0
10   Tennis Court 2    150.0
11   Tennis Court 1    150.0
12     Squash Court    140.0
13     Squash Court    105.0
14  Badminton Court     93.0
15   Tennis Court 1     75.0
16   Tennis Court 2     75.0
17     Squash Court     70.0
18  Badminton Court     46.5
19   Tennis Court 1     45.0
20   Tennis Court 2     45.0
21   Massage Room 1     39.6
22     Squash Court     35.0
23   Tennis Court 1     30.0
24   Tennis Court 2     30.0
25     Squash Court     21.0
26    Snooker Table     20.0
27   Massage Room 2     19.8
28   Massage Room 1     19.8
29   Tennis Court 2     15.0
30   Tennis Court 1     15.0
31     Squash Court     14.0
32       Pool Table     10.0
33    Snooker Table     10.0
34     Table Tennis     10.0
35     Squash Court      7.0
36       Pool Table      5.0
37  Badminton Court      0.0
38     Table Tennis      0.0
39       Pool Table      0.0
40    Snooker Table      0.0